# 🧑‍🏫 Aula 3 - RP com Visão: Avaliação do Modelo e Definição de Limiares

---

## 📚 Sumário

1. [Introdução](#1-introdução)
2. [Métricas de Avaliação do Azure](#2-métricas-de-avaliação-do-azure)
3. [Teste com Novas Imagens (Conjunto de Teste)](#3-teste-com-novas-imagens-conjunto-de-teste)
4. [Análise de Limiares (Threshold)](#4-análise-de-limiares-threshold)
5. [Publicação e Consumo do Modelo](#5-publicação-e-consumo-do-modelo)

---

## 🎯 Capacidades Técnicas Trabalhadas

- **Utilizar método de classificação supervisionada** para reconhecimento de padrões.
- **Selecionar o método mais adequado** para o reconhecimento de padrões.

## 🤝 Capacidades Socioemocionais Trabalhadas

- **Ética**: Apresentar comportamento ético na conduta profissional.
- **Resolução de problemas complexos**.

---

## 1. Introdução

Na aula anterior, treinamos o modelo. Agora precisamos saber: **ele é bom o suficiente?**

Vamos analisar:
- **Precision (Precisão)**: Quando o modelo diz que é "Defeito X", ele acerta?
- **Recall (Revocação)**: De todos os "Defeitos X" que existem, quantos o modelo encontrou?
- **Matriz de Confusão**: Onde o modelo está errando?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install azure-cognitiveservices-vision-customvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from PIL import Image

# Bibliotecas do Azure (necessário ter instalado na Aula 2)
try:
    from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
    from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
    from msrest.authentication import ApiKeyCredentials
    print("✅ SDK do Azure carregada.")
except ImportError:
    print("⚠️ SDK do Azure não instalada. Execute: pip install azure-cognitiveservices-vision-customvision")

✅ SDK do Azure carregada.


---

## 2. Métricas de Avaliação do Azure

O próprio Azure fornece métricas da iteração treinada. Podemos acessá-las via código.

In [ ]:
# Configurações
ENDPOINT = "https://custom3dprinterbrandao.cognitiveservices.azure.com/"
ENDPOINT_PREDICTION = "https://custom3dprinterbrandao-Prediction.cognitiveservices.azure.com/"
TRAINING_KEY = "<REDACTED_TRAINING_KEY>"
PREDICTION_KEY = "<REDACTED_PREDICTION_KEY>" # Necessária para teste
PREDICTION_RESOURCE_ID = "custom3dprinterbrandao-Prediction"
PROJECT_ID = "33ee74bf-84e1-4d52-ba35-798ba4a41fc8" # ID do projeto criado na aula 2

def get_metrics(trainer, project_id):
    if trainer is None: return

    # Pega a última iteração
    iterations = trainer.get_iterations(project_id)
    latest_iteration = iterations[0]

    # Pega performance
    performance = trainer.get_iteration_performance(project_id, latest_iteration.id)

    print(f"📊 Performance da Iteração: {latest_iteration.name}")
    print(f"   Precision: {performance.precision:.2%}")
    print(f"   Recall:    {performance.recall:.2%}")
    print(f"   mAP:       {performance.average_precision:.2%}")

    # Performance por Tag
    print("\n📌 Por Classe:")
    for tag_perf in performance.per_tag_performance:
        print(f"   - {tag_perf.name}: P={tag_perf.precision:.2f}, R={tag_perf.recall:.2f}")

# Exemplo de uso:
credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
get_metrics(trainer, PROJECT_ID)

📊 Performance da Iteração: Iteration 1
   Precision: 100.00%
   Recall:    57.85%
   mAP:       96.13%

📌 Por Classe:
   - bed_no_stick: P=1.00, R=0.40
   - leg_broken: P=1.00, R=0.48
   - no_bottom: P=0.00, R=0.00
   - no_defected: P=1.00, R=0.60
   - no_support: P=1.00, R=0.73


In [ ]:
iterations = trainer.get_iterations(PROJECT_ID)
latest_iteration = iterations[0]

ITERATION_ID = latest_iteration.id
PUBLISH_ITERATION_NAME = latest_iteration.name

print(ITERATION_ID)

print(f"Publishing the current iteration as '{PUBLISH_ITERATION_NAME}'...")
trainer.publish_iteration(
    PROJECT_ID,
    ITERATION_ID,
    PUBLISH_ITERATION_NAME,
    PREDICTION_RESOURCE_ID
)

03d5d68a-9acb-4330-b705-1abb334be35d
Publishing the current iteration as 'Iteration 1'...


CustomVisionErrorException: Iteration is already published as: Iteration1

---

## 3. Teste com Novas Imagens (Conjunto de Teste)

Vamos usar o Custom Vision **Prediction Client** para enviar as imagens que separamos na pasta `processed_data/vision/test` e ver o resultado real.

In [ ]:
# Publicar iteração antes de testar (se ainda não estiver)
import time

def test_model(prediction_client, project_id, test_dir, threshold=0.5):
    if prediction_client is None: return

    # O nome da publicação costuma ser "Iteration1" ou similar
    PUBLISH_NAME = "Iteration1"

    print(f"Testando imagens de {test_dir}...")

    results = []

    for folder in test_dir.iterdir():
        if folder.is_dir():
            true_label = folder.name
            images = list(folder.glob("*.jpg"))[:5] # Testar 5 de cada para economizar tempo/quota

            for img_path in images:
                with open(img_path, "rb") as image_contents:
                    prediction = prediction_client.classify_image(
                        project_id,
                        PUBLISH_NAME,
                        image_contents.read()
                    )

                # Pega a melhor predição
                best_pred = prediction.predictions[0]

                # Aplica filtro de threshold
                pred_label = best_pred.tag_name if best_pred.probability >= threshold else "Uncertain"

                results.append({
                    "image": img_path.name,
                    "true": true_label,
                    "pred": pred_label,
                    "prob": best_pred.probability
                })
                print(f"Img: {img_path.name} | Real: {true_label} -> Pred: {pred_label} ({best_pred.probability:.2%})")
                import time
                time.sleep(1)
    return results

# Exemplo de uso:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT_PREDICTION, prediction_credentials)
TEST_DIR = Path("/content/drive/MyDrive/Colab Notebooks/Visão Computacional/Azure - Custom Vision/3D-Printer-Avaliação")
results = test_model(predictor, PROJECT_ID, TEST_DIR)

03d5d68a-9acb-4330-b705-1abb334be35d
Publishing the current iteration as 'Iteration1'...


CustomVisionErrorException: Invalid prediction resource id

---

## 4. Análise de Limiares (Threshold)

O modelo retorna uma probabilidade (0 a 1). Precisamos decidir a partir de quanto confiamos na resposta.

- **Threshold Alto (>0.8)**: Menos falsos positivos, mas podemos perder defeitos reais (Falso Negativo).
- **Threshold Baixo (<0.5)**: Detectamos tudo, mas muito alarme falso.

### Exemplo Prático
Se o modelo diz "leg_broken" com 40% de certeza, devemos parar a impressora?

> **Discussão:** Em manufatura, errar um defeito (deixar passar) é pior do que um falso alarme? Ou parar a linha toda hora por falso alarme é inviável?

In [ ]:
# Simulação de impacto de threshold
probs = [0.95, 0.82, 0.60, 0.45, 0.10]
print("Probabilidades de defeito encontradas:", probs)

for t in [0.5, 0.7, 0.9]:
    detected = [p for p in probs if p >= t]
    print(f"Threshold {t}: {len(detected)} detectados")

Probabilidades de defeito encontradas: [0.95, 0.82, 0.6, 0.45, 0.1]
Threshold 0.5: 3 detectados
Threshold 0.7: 2 detectados
Threshold 0.9: 1 detectados


---

## 5. Publicação e Consumo do Modelo

Para integrar com o Sistema Baseado em Conhecimento (SBC), o modelo precisa expor uma API.
No Azure, isso é feito clicando em **Publish**.

A saída para o SBC é um pacote de "Fatos":
```json
{
  "source": "vision_module",
  "detected_class": "leg_broken",
  "confidence": 0.92,
  "timestamp": "2024-02-14T10:00:00"
}
```